This notebook is a template on using Masked Language Modeling to identify datasets in papers.

The training of the Bert model was done in another notebook: \[Coleridge\] BERT - Masked Dataset Modeling.

The approach is:
- Locate all the sequences of capitalized words (these sequences may contain some stopwords),
- Replace each sequence with one of 2 special symbols (e.g. $ and #), implying if that sequence represents a dataset name or not.
- Have the model learn the MLM task.

# Install packages

In [1]:
!pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
!pip install ../input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
!pip install ../input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl

Looking in links: file:///kaggle/input/coleridge-packages/packages/datasets
Processing /kaggle/input/coleridge-packages/packages/datasets/datasets-1.5.0-py3-none-any.whl
Processing /kaggle/input/coleridge-packages/packages/datasets/huggingface_hub-0.0.7-py3-none-any.whl
Processing /kaggle/input/coleridge-packages/packages/datasets/tqdm-4.49.0-py2.py3-none-any.whl
Processing /kaggle/input/coleridge-packages/packages/datasets/xxhash-2.0.0-cp37-cp37m-manylinux2010_x86_64.whl
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.56.2
    Uninstalling tqdm-4.56.2:
      Successfully uninstalled tqdm-4.56.2
Processing /kaggle/input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
Processing /kaggle/input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
tokenizers is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/coleridge-packages/transformers-4.5.0.dev0-

# Import

In [2]:
import os
import re
import json
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, \
AutoModelForMaskedLM, Trainer, TrainingArguments, pipeline

sns.set()
random.seed(123)
np.random.seed(456)
torch.manual_seed(2021)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
PRETRAINED_PATH = '../input/coleridge-bert-masked-dataset-modeling/mlm-model'
TOKENIZER_PATH = '../input/coleridge-bert-masked-dataset-modeling/model_tokenizer'

MAX_LENGTH = 30
OVERLAP = 20

PREDICT_BATCH = 32

DATASET_SYMBOL = '$' # this symbol represents a dataset name
NONDATA_SYMBOL = '#' # this symbol represents a non-dataset name

# Load data

In [4]:
# sample_submission_path = '../input/coleridgeinitiative-show-us-the-data/sample_submission.csv'
# sample_submission = pd.read_csv(sample_submission_path)

# sample_submission = sample_submission[:1]

# paper_test_folder = '../input/coleridgeinitiative-show-us-the-data/test'
# papers = {}
# for paper_id in sample_submission['Id']:
#     with open(f'{paper_test_folder}/{paper_id}.json', 'r') as f:
#         paper = json.load(f)
#         papers[paper_id] = paper
        
# train
sample_submission_path = '../input/coleridgeinitiative-show-us-the-data/train.csv'
paper_test_folder = '../input/coleridgeinitiative-show-us-the-data/train'

sample_submission = pd.read_csv(sample_submission_path)
sample_submission = sample_submission[:8000]
# Group by publication, training labels should have the same form as expected output.
sample_submission = sample_submission.groupby('Id').agg({
    'pub_title': 'first',
    'dataset_title': '|'.join,
    'dataset_label': '|'.join,
    'cleaned_label': '|'.join
}).reset_index()    

print('sample_submission: ', len(sample_submission))


papers = {}
for paper_id in sample_submission['Id']:
    with open(f'{paper_test_folder}/{paper_id}.json', 'r') as f:
        paper = json.load(f)
        papers[paper_id] = paper

sample_submission:  7379


In [5]:
sample_submission = sample_submission[6379:]

# Transform data to MLM format

### Load model and tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH, use_fast=True)
model = AutoModelForMaskedLM.from_pretrained(PRETRAINED_PATH)

mlm = pipeline(
    'fill-mask', 
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

### Auxiliary functions

In [7]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def jaccard_similarity(s1, s2):
    l1 = s1.split(" ")
    l2 = s2.split(" ")    
    intersection = len(list(set(l1).intersection(l2)))
    union = (len(l1) + len(l2)) - intersection
    return float(intersection) / union

def clean_paper_sentence(s):
    """
    This function is essentially clean_text without lowercasing.
    """
    s = re.sub('[^A-Za-z0-9]+', ' ', str(s)).strip()
    s = re.sub(' +', ' ', s)
    return s

def shorten_sentences(sentences):
    """
    Sentences that have more than MAX_LENGTH words will be split
    into multiple sentences with overlappings.
    """
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

connection_tokens = {'s', 'of', 'and', 'in', 'on', 'for', 'data', 'dataset'}
def find_mask_candidates(sentence):
    """
    Extract masking candidates for Masked Dataset Modeling from a given $sentence.
    A candidate should be a continuous sequence of at least 2 words, 
    each of these words either has the first letter in uppercase or is one of
    the connection words ($connection_tokens). Furthermore, the connection 
    tokens are not allowed to appear at the beginning and the end of the
    sequence.
    """
    def candidate_qualified(words):
        while len(words) and words[0].lower() in connection_tokens:
            words = words[1:]
        while len(words) and words[-1].lower() in connection_tokens:
            words = words[:-1]
        
        return len(words) >= 2
    
    candidates = []
    
    phrase_start, phrase_end = -1, -1
    for id in range(1, len(sentence)):
        word = sentence[id]
        if word[0].isupper() or word in connection_tokens:
            if phrase_start == -1:
                phrase_start = phrase_end = id
            else:
                phrase_end = id
        else:
            if phrase_start != -1:
                if candidate_qualified(sentence[phrase_start:phrase_end+1]):
                    candidates.append((phrase_start, phrase_end))
                phrase_start = phrase_end = -1
    
    if phrase_start != -1:
        if candidate_qualified(sentence[phrase_start:phrase_end+1]):
            candidates.append((phrase_start, phrase_end))
    
    return candidates

### Transform

In [8]:
mask = mlm.tokenizer.mask_token

In [9]:
mask

'[MASK]'

In [10]:
all_test_data = []

# pbar = tqdm(total = len(sample_submission))
for paper_id in sample_submission['Id']:
    # load paper
    paper = papers[paper_id]
    
    # extract sentences
    sentences = set([clean_paper_sentence(sentence) for section in paper 
                     for sentence in section['text'].split('.')
                    ])
    sentences = shorten_sentences(sentences) # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    sentences = [sentence for sentence in sentences if any(word in sentence.lower() for word in ['data', 'study'])]
    sentences = [sentence.split() for sentence in sentences] # sentence = list of words
    
    # mask
    test_data = []
    for sentence in sentences:
        for phrase_start, phrase_end in find_mask_candidates(sentence):
            dt_point = sentence[:phrase_start] + [mask] + sentence[phrase_end+1:]
            test_data.append((' '.join(dt_point), ' '.join(sentence[phrase_start:phrase_end+1]))) # (masked text, phrase)
    
    all_test_data.append(test_data)
    
    # process bar
#     pbar.update(1)

### Predict

In [11]:
pred_labels = []

pbar = tqdm(total = len(all_test_data))
for test_data in all_test_data:
    pred_bag = set()
    
    if len(test_data):
        texts, phrases = list(zip(*test_data))
        mlm_pred = []
        for p_id in range(0, len(texts)):
            batch_texts = texts #[p_id:p_id+PREDICT_BATCH]
            batch_pred = mlm(list(batch_texts), targets=[f' {DATASET_SYMBOL}', f' {NONDATA_SYMBOL}'])
            
            if len(batch_texts) == 1:
                batch_pred = [batch_pred]
            
            mlm_pred.extend(batch_pred)
        
        for (result1, result2), phrase in zip(mlm_pred, phrases):
            if (result1['score'] > result2['score']*1.5 and result1['token_str'] == DATASET_SYMBOL) or\
               (result2['score'] > result1['score']*1.5 and result2['token_str'] == NONDATA_SYMBOL):
                pred_bag.add(clean_text(phrase))
    
    # filter labels by jaccard score 
    filtered_labels = []
    
    for label in sorted(pred_bag, key=len, reverse=True):
        if len(filtered_labels) == 0 or all(jaccard_similarity(label, got_label) < 0.75 for got_label in filtered_labels):
            filtered_labels.append(label)
            
    pred_labels.append('|'.join(filtered_labels))
    pbar.update(1)

sample_submission['PredictionString'] = pred_labels
sample_submission.to_csv('submission.csv', index=False)

100%|██████████| 1000/1000 [32:39<00:00,  1.18s/it]

In [12]:
sample_submission['PredictionString'].head().to_list()

['adni and cache county data|cache county',
 '',
 '',
 'prostate cancer prevention trial',
 'european genome wide association study gwas']

In [13]:
from typing import List
import string
from functools import partial
import numpy as np

In [14]:
# https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/discussion/230091
def compute_fbeta(y_true: List[List[str]],
                  y_pred: List[List[str]],
                  beta: float = 0.5) -> float:
    """Compute the Jaccard-based micro FBeta score.

    References
    ----------
    - https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/overview/evaluation
    """

    def _jaccard_similarity(str1: str, str2: str) -> float:
        a = set(str1.split()) 
        b = set(str2.split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))

    tp = 0  # true positive
    fp = 0  # false positive
    fn = 0  # false negative
    for ground_truth_list, predicted_string_list in zip(y_true, y_pred):
        predicted_string_list_sorted = sorted(predicted_string_list)
        for ground_truth in sorted(ground_truth_list):            
            if len(predicted_string_list_sorted) == 0:
                fn += 1
            else:
                similarity_scores = [
                    _jaccard_similarity(ground_truth, predicted_string)
                    for predicted_string in predicted_string_list_sorted
                ]
                matched_idx = np.argmax(similarity_scores)
                if similarity_scores[matched_idx] >= 0.5:
                    predicted_string_list_sorted.pop(matched_idx)
                    tp += 1
                else:
                    fn += 1
        fp += len(predicted_string_list_sorted)

    tp *= (1 + beta ** 2)
    fn *= beta ** 2
    fbeta_score = tp / (tp + fp + fn)
    return fbeta_score

In [15]:
COMPUTE_CV = True
if COMPUTE_CV:
    COMPUTE_CV_SCORE = compute_fbeta(sample_submission['cleaned_label'].apply(lambda x: [x]),\
                  sample_submission['PredictionString'].apply(lambda x: x.split('|')))
    print('COMPUTE_CV_SCORE =', COMPUTE_CV_SCORE)
else:
    print(f'COMPUTE_CV = {COMPUTE_CV}')
    
# print(f'ALL_BLENDED = {ALL_BLENDED}')
# print(f'BASELINE_HELPING = {BASELINE_HELPING}')
# print(f'MATCH_ONLY = {MATCH_ONLY}')

COMPUTE_CV_SCORE = 0.11266447368421052


In [16]:
sample_submission[:5]

,Id,pub_title,dataset_title,dataset_label,cleaned_label,PredictionString
6379,dce3207b-8aac-4b80-8f9b-2657feb1a810,Common DNA Variants Accurately Rank an Individ...,Alzheimer's Disease Neuroimaging Initiative (A...,ADNI,adni,adni and cache county data|cache county
6380,dceca720-75f9-4c91-b6e4-0f28467167c2,From Sputnik to PISA Shock – New Technology an...,Trends in International Mathematics and Scienc...,Trends in International Mathematics and Scienc...,trends in international mathematics and scienc...,
6381,dcf661da-9b5d-4d1f-9062-5394a38ee067,Identifying Factors Affecting the Quality of T...,Trends in International Mathematics and Scienc...,Trends in International Mathematics and Scienc...,trends in international mathematics and scienc...,
6382,dcfdba13-e25b-40ce-9747-b3fe3a31a334,Should prostate-specific antigen velocity be a...,Baltimore Longitudinal Study of Aging (BLSA),Baltimore Longitudinal Study of Aging,baltimore longitudinal study of aging,prostate cancer prevention trial
6383,dd024565-6d8d-45c2-ac04-1e454abbf409,Positioning Personal Polygenic Risk score agai...,Alzheimer's Disease Neuroimaging Initiative (A...,ADNI,adni,european genome wide association study gwas
